In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
import csv


In [2]:
data = pd.read_csv("../Data/Segmented_Data/Porosity/XCT-1.csv")

data = data.sort_values(by=["Volume (µm³)"], ascending=False)

data = data.head(5)

data = pd.DataFrame(data)
data

,Time Step,Label Index,Name (NA),Voxel count,Volume (µm³),Equivalent Spherical Diameter (µm),Aspect Ratio,Center Of Mass X (µm),Center Of Mass Y (µm),Center Of Mass Z (µm),Particle Radius Min (µm),Particle Radius Max (µm),Particle Radius Mean (µm),Sphericity
0,0,16271,NaN,5685.0,5685000.0,221.433818,0.421967,11757.604222,5752.311346,1449.503958,13.615896,233.554758,139.211343,0.593017
1,0,16270,NaN,4770.0,4770000.0,208.852616,0.227072,11777.731656,9234.844864,766.593291,31.238707,270.320929,146.698368,0.579101
2,0,16269,NaN,3949.0,3949000.0,196.108388,0.412162,11774.226386,7806.416814,305.183591,8.222153,246.723765,136.982670,0.467630
3,0,16268,NaN,3728.0,3728000.0,192.379634,0.279082,11741.630901,4936.810622,7732.752146,16.752433,256.998802,129.405736,0.568906
4,0,16267,NaN,3396.0,3396000.0,186.490345,0.158551,11761.269140,7011.722615,4574.243227,14.257371,348.657198,150.275820,0.488714


In [3]:
QR_data_absolute = pd.read_csv("../Data/Segmented_Data/Porosity/XCT-3.csv", usecols = ["Center Of Mass X (µm)", "Center Of Mass Y (µm)", "Center Of Mass Z (µm)"])

QR_data_absolute = QR_data_absolute.head(5)

pair_dist = QR_data_absolute.to_numpy()

pair_distance = np.round(pdist(pair_dist), 10)   

matrix_dist_vol = np.zeros(6*len(pair_distance))

#print(len(matrix_dist_vol))

Volume_pores = np.array(data["Volume (µm³)"])

#print(Volume_pores)

matrix_dist_vol = matrix_dist_vol.reshape(-1, 6)

pores_array = np.arange(0, len(data))

#print(pores_array)

list_1 = pores_array
list_2 = pores_array
 
unique_combinations = []
 
for i in range(len(list_1)):
    for j in range(len(list_2)):
        if i >= j:
            continue
        unique_combinations.append([0., 0., 0., 0., list_1[i], list_2[j]])

unique_combinations = np.array(unique_combinations)

#print("3d array: ", unique_combinations)

for i, dist_value in enumerate(pair_distance):
    index_volume_1 = unique_combinations[i][4] 
    index_volume_2 = unique_combinations[i][5]
    #print(index_volume_1)
    #print(Volume_pores[index_volume_2])
    matrix_dist_vol[i][0] = index_volume_1 + 1
    matrix_dist_vol[i][1] = index_volume_2 + 1
    matrix_dist_vol[i][2] = pair_distance[i]
    #print(Volume_pores[int(index_volume_1)])
    matrix_dist_vol[i][3] = Volume_pores[int(index_volume_1)]
    matrix_dist_vol[i][4] = Volume_pores[int(index_volume_2)]

matrix_dist_vol[:,5] = matrix_dist_vol[:,3] + matrix_dist_vol[:,4]

#print("Matrix ", matrix_dist_vol)
# export as csv file
dist_vol_data = {'Index 1':matrix_dist_vol[:,0], 'Index 2':matrix_dist_vol[:,1],'Pair-wise Distance (µm)':matrix_dist_vol[:,2], 'Volume A (µm³)':matrix_dist_vol[:,3], 'Volume B (µm³)':matrix_dist_vol[:,4], 'Pair-wise Volume (µm³)':matrix_dist_vol[:,5]}

df_pair_distance = pd.DataFrame(dist_vol_data)
df_pair_distance.index += 1

df_pair_distance = df_pair_distance.sort_values(by=["Pair-wise Volume (µm³)"], ascending=False)

df_pair_distance.iloc[:, 0] = df_pair_distance.iloc[:, 0].astype(int)
df_pair_distance.iloc[:, 1] = df_pair_distance.iloc[:, 1].astype(int)
df_pair_distance.iloc[:, 2] = df_pair_distance.iloc[:, 2].astype(int)
df_pair_distance.iloc[:, 3] = df_pair_distance.iloc[:, 3].astype(int)
df_pair_distance.iloc[:, 4] = df_pair_distance.iloc[:, 4].astype(int)
df_pair_distance.iloc[:, 5] = df_pair_distance.iloc[:, 5].astype(int)

df_pair_distance.to_csv("../Data/Processed_Data/Uniquness/XCT-3.csv", index=False)

C:\Users\kgupta\AppData\Local\Temp\ipykernel_29520\144973775.py:61: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_pair_distance.iloc[:, 0] = df_pair_distance.iloc[:, 0].astype(int)
C:\Users\kgupta\AppData\Local\Temp\ipykernel_29520\144973775.py:62: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_pair_distance.iloc[:, 1] = df_pair_distance.iloc[:, 1].astype(int)
C:\Users\kgupta\AppData\Local\Temp\ipykernel_29520\144973775.py:63: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always s

In [4]:
df_pair_distance


,Index 1,Index 2,Pair-wise Distance (µm),Volume A (µm³),Volume B (µm³),Pair-wise Volume (µm³)
1,1,2,13980,5685000,4770000,10455000
2,1,3,11967,5685000,3949000,9634000
3,1,4,16683,5685000,3728000,9413000
4,1,5,13789,5685000,3396000,9081000
5,2,3,5450,4770000,3949000,8719000
6,2,4,6131,4770000,3728000,8498000
7,2,5,6783,4770000,3396000,8166000
8,3,4,4738,3949000,3728000,7677000
9,3,5,2223,3949000,3396000,7345000
10,4,5,3511,3728000,3396000,7124000
